In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Mon Oct 27 12:56:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8       

In [2]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
%%writefile validate_mpgguf.cu

// mpgguf_validate.cu  — MPGGUF v2 validator with hardened GGUF/MP readers
//
// Build:
//   nvcc -O3 -std=c++17 -o mpgguf_validate mpgguf_validate.cu
//
// Usage:
//   ./mpgguf_validate --mpgguf model.mpgguf --fp16 baseline_f16_or_f32.gguf [--report] [--diff-high-low]
//
// What it does:
//   * Loads MPGGUF v2 container (LOW/HIGH/FP slots, adjacent packing).
//   * Loads baseline GGUF (FP16 or FP32 weights) using a tolerant KV skipper and safe strings.
//   * Dequantizes HIGH (Q8_0) and legacy scalar 2-bit LOW when recognizable by size.
//   * Computes MSE, RMSE, and max|Δ| vs FP16 baseline (and optional HIGH vs LOW).
//   * Skips unknown 2-bit layouts (e.g., Q2_K / IQ2_XXS) with a clear notice.
//
// Notes:
//   * No inference — this only validates dequantized weights vs a baseline.
//   * All readers are fast-fail hardened to avoid endless parsing on malformed files.

#include <cuda_fp16.h>
#include <cuda_runtime.h>
#include <stdint.h>

#include <algorithm>
#include <cmath>
#include <cstring>
#include <fstream>
#include <iostream>
#include <sstream>
#include <string>
#include <unordered_map>
#include <vector>

// ------------- CUDA helpers -------------
#define CUDA_OK(x)                                                                                          \
    do {                                                                                                    \
        auto _e = (x);                                                                                      \
        if (_e != cudaSuccess) {                                                                            \
            std::cerr << "CUDA " << cudaGetErrorString(_e) << " @ " << __FILE__ << ":" << __LINE__ << "\n"; \
            std::exit(1);                                                                                   \
        }                                                                                                   \
    } while (0)

#ifndef QK
#    define QK 32  // scalar block size used by Q8_0 and the legacy 2-bit variants
#endif

static inline bool aligned32(uint64_t x) {
    return (x & 31ull) == 0ull;
}

static inline bool aligned64(uint64_t x) {
    return (x & 63ull) == 0ull;
}


static std::vector<uint8_t> readStreamData(std::ifstream& f, size_t sz, size_t offset) {
    std::vector<uint8_t> out;

    if (!f.good()) return out;

    // 1) Clear any eof/fail from prior ops; required before seekg on some FUSE filesystems.
    f.clear();

    // 2) Find file size.
    std::istream::pos_type cur = f.tellg();
    f.seekg(0, std::ios::end);
    std::istream::pos_type end = f.tellg();
    if (end <= 0) { f.clear(); return out; }
    const size_t file_size = static_cast<size_t>(end);

    // 3) Validate and clamp request.
    if (offset >= file_size) { f.clear(); return out; }
    const size_t to_read = std::min(sz, file_size - offset);

    // 4) Seek to absolute offset from beginning.
    f.clear(); // (again, in case tellg/setg flipped eof)
    f.seekg(static_cast<std::streamoff>(offset), std::ios::beg);
    if (!f.good()) { f.clear(); return out; }

    // 5) Read with partial-read handling.
    out.resize(to_read);
    f.read(reinterpret_cast<char*>(out.data()), static_cast<std::streamsize>(to_read));
    const auto got = static_cast<size_t>(f.gcount());
    if (got < to_read) {
        out.resize(got);
        // Reset state so later calls can still seek/read.
        f.clear();
    }
    return out;
}


static uint64_t align_up(uint64_t off, uint32_t align)
{
    const uint64_t mask = static_cast<uint64_t>(align) - 1u;
    return (off + mask) & ~mask;
}

static inline bool is_ascii_identifier(const std::string& s) {
    if (s.empty() || s.size() > 512)
        return false;

    for (unsigned char c : s)
    {
        if (!((c >= 'a' && c <= 'z') || (c >= 'A' && c <= 'Z') || (c >= '0' && c <= '9') || c == '_' || c == '.' ||
            c == '/' || c == '-')) {
            return false;
        }
    }
    return true;
}

//=============== MPGGUF v2 reader ===============
struct Rec
{
    std::string           name;
    uint32_t              nd;
    std::vector<uint64_t> dims;
    uint32_t              flags, g_low, g_high, g_fp;
    uint64_t              off_low, sz_low, off_high, sz_high, off_fp, sz_fp;
};

struct MP
{
    std::vector<Rec>     recs;
    std::vector<uint8_t> kv;
    std::ifstream        f;
    size_t               data_offset;
    size_t               data_sz;

    bool Open(const std::string& path)
    {
        f.open(path, std::ios::binary);
        if (!f.is_open()) {
            return false;
        }

        return true;
    }
};

static inline uint32_t rd_u32(const uint8_t* p)
{
    uint32_t v;
    memcpy(&v, p, 4);
    return v;
}

static inline uint64_t rd_u64(const uint8_t* p)
{
    uint64_t v;
    memcpy(&v, p, 8);
    return v;
}

static bool in_range(uint64_t off, uint64_t sz, size_t total)
{
    return off <= (uint64_t)total && sz <= (uint64_t)total && (off + sz) <= (uint64_t)total;
}

bool load_mp(const std::string& path, MP& out)
{
    if (out.Open(path) == false)
        return false;

    auto& f = out.f;

    // Header
    char mg[7];
    f.read(mg, 7);
    uint8_t ver = 0;
    f.read((char*)&ver, 1);
    if (std::string(mg, 7) != "MPGGUF2" || ver != 2) {
        std::cerr << "Not MPGGUF2 (" << path << ")\n";
        return false;
    }

    uint64_t kvsz = 0;
    uint32_t nt = 0;
    f.read((char*)&kvsz, 8);
    f.read((char*)&nt, 4);

    // Sanity cap on tensor count
    const uint64_t kMaxT = 200000;
    if (nt == 0 || nt > kMaxT)
    {
        std::cerr << "ERROR: suspicious mpgguf n_t=" << nt << "\n";
        return false;
    }

    // Directory
    out.recs.reserve(nt);
    const uint32_t kMaxName = 4096;
    for (uint32_t i = 0; i < nt; i++)
    {
        uint32_t nl = 0;
        f.read((char*)&nl, 4);
        if (nl == 0 || nl > kMaxName)
        {
            std::cerr << "ERROR: mpgguf name length " << nl << "\n";
            return false;
        }
        std::string name(nl, '\0');
        f.read(name.data(), nl);
        if (!is_ascii_identifier(name))
        {
            std::cerr << "ERROR: mpgguf bad name " << name << "\n";
            return false;
        }

        uint32_t nd = 0;
        f.read((char*)&nd, 4);
        if (nd == 0 || nd > 6)
        {
            std::cerr << "ERROR: mpgguf bad nd=" << nd << " for " << name << "\n";
            return false;
        }
        std::vector<uint64_t> dims(nd);
        for (uint32_t d = 0; d < nd; ++d)
        {
            f.read((char*)&dims[d], 8);
            if (dims[d] == 0 || dims[d] > (uint64_t) 1e10)
            {
                std::cerr << "ERROR: mpgguf bad dim[" << d << "]=" << dims[d] << " for " << name << "\n";
                return false;
            }
        }

        uint32_t flags = 0, gL = 0, gH = 0, gF = 0;
        f.read((char*)&flags, 4);
        f.read((char*)&gL, 4);
        f.read((char*)&gH, 4);
        f.read((char*)&gF, 4);
        if ((flags & ~0x7u) != 0)
        {
            std::cerr << "ERROR: mpgguf flags reserved bits set for " << name << "\n";
            return false;
        }

        uint64_t oL = 0, sL = 0, oH = 0, sH = 0, oF = 0, sF = 0;
        f.read((char*)&oL, 8);
        f.read((char*)&sL, 8);
        f.read((char*)&oH, 8);
        f.read((char*)&sH, 8);
        f.read((char*)&oF, 8);
        f.read((char*)&sF, 8);

        if (flags & 0x1)
        {
            if (!aligned64(oL))
            {
                std::cerr << "ERROR: LOW off not 64B aligned\n";
                return false;
            }
        }
        if (flags & 0x2)
        {
            if (!aligned64(oH)) {
                std::cerr << "ERROR: HIGH off not 64B aligned\n";
                return false;
            }
        }
        if (flags & 0x4)
        {
            if (!aligned64(oF)) {
                std::cerr << "ERROR: FP off not 64B aligned\n";
                return false;
            }
        }

        out.recs.push_back({ std::move(name), nd, std::move(dims), flags, gL, gH, gF, oL, sL, oH, sH, oF, sF });
    }

    // KV
    out.kv.resize(kvsz);
    if (kvsz)
    {
        f.read((char*)out.kv.data(), kvsz);
    }

    // Data region
    // 1. Get the current position
    std::streampos current_pos = f.tellg();
    current_pos = align_up(current_pos, 64);

    // 2. Seek to the end
    f.seekg(0, std::ios::end);

    // 3. Get the end position
    std::streampos end_pos = f.tellg();

    // 4. Calculate remaining size
    std::streamsize remaining_size = end_pos - current_pos;

    // 5. Seek back to the current position
    f.seekg(current_pos);

    out.data_offset = (size_t)current_pos;
    out.data_sz = (size_t)remaining_size;

    return true;
}

// =============== GGUF (baseline) reader ===============
enum
{
    GGUF_U8 = 0,
    GGUF_I8 = 1,
    GGUF_U16 = 2,
    GGUF_I16 = 3,
    GGUF_U32 = 4,
    GGUF_I32 = 5,
    GGUF_F32 = 6,
    GGUF_BOOL = 7,
    GGUF_STRING = 8,
    GGUF_ARRAY = 9,
    GGUF_U64 = 10,
    GGUF_I64 = 11,
    GGUF_F64 = 12,
    GGUF_BYTES = 13
};

static int scalar_size(int t)
{
    switch (t) {
    case GGUF_U8:
    case GGUF_I8:
    case GGUF_BOOL:
        return 1;
    case GGUF_U16:
    case GGUF_I16:
        return 2;
    case GGUF_U32:
    case GGUF_I32:
    case GGUF_F32:
        return 4;
    case GGUF_U64:
    case GGUF_I64:
    case GGUF_F64:
        return 8;
    default:
        return -1;
    }
}

static std::string rd_s(std::istream& f)
{
    uint64_t n = 0;
    f.read((char*)&n, 8);
    const uint64_t kMaxStr = 4096 * 1024;  // cap for safety
    if (n > kMaxStr) {
        throw std::runtime_error("gguf string length over cap");
    }
    std::string s((size_t)n, '\0');
    if (n) {
        f.read(s.data(), (std::streamsize)n);
    }
    return s;
}

static void skip_bytes(std::istream& f)
{
    uint64_t n = 0;
    f.read((char*)&n, 8);
    f.seekg((std::streamoff)n, std::ios::cur);
}

static void skipv(std::istream& f)
{
    uint32_t t = 0;
    f.read((char*)&t, 4);
    int sz = scalar_size((int)t);
    if (sz > 0) {
        f.seekg(sz, std::ios::cur);
        return;
    }
    if (t == GGUF_STRING) {
        (void)rd_s(f);
        return;
    }
    if (t == GGUF_BYTES) {
        skip_bytes(f);
        return;
    }
    if (t == GGUF_ARRAY) {
        uint32_t et = 0;
        uint64_t cnt = 0;
        f.read((char*)&et, 4);
        f.read((char*)&cnt, 8);
        int es = scalar_size((int)et);
        if (es > 0) {
            f.seekg((std::streamoff)es * (std::streamoff)cnt, std::ios::cur);
            return;
        }
        if (et == GGUF_STRING) {
            for (uint64_t i = 0; i < cnt; i++) {
                (void)rd_s(f);
            }
            return;
        }
        if (et == GGUF_BYTES) {
            for (uint64_t i = 0; i < cnt; i++) {
                skip_bytes(f);
            }
            return;
        }
        if (et == GGUF_ARRAY) {
            for (uint64_t i = 0; i < cnt; i++) {
                skipv(f);
            }
            return;
        }
        for (uint64_t i = 0; i < cnt; i++) {
            skip_bytes(f);  // fallback
        }
        return;
    }
    skip_bytes(f);  // unknown top-level -> treat as BYTES
}

struct GRec
{
    size_t                splitId;
    std::string           name;
    uint32_t              nd;
    std::vector<uint64_t> dims;
    uint32_t              g;
    uint64_t              off, sz;
};

struct GG
{
    std::vector<GRec>    recs;
    std::ifstream f;

    bool Open(const std::string& path)
    {
        f.open(path, std::ios::binary);
        if (!f.is_open()) {
            return false;
        }

        return true;
    }
};

static size_t numel(const std::vector<uint64_t>& d)
{
    size_t n = 1;
    for (auto v : d) {
        n *= size_t(v);
    }
    return n;
}

bool load_fp(const size_t splitId, const std::string& path, GG& out)
{
    if (!out.Open(path))
    {
        std::cerr << "open failed: " << path << "\n";
        return false;
    }

    char mg[4];
    out.f.read(mg, 4);
    if (std::string(mg, 4) != "GGUF") {
        std::cerr << "not GGUF: " << path << "\n";
        return false;
    }
    uint32_t ver = 0;
    out.f.read((char*)&ver, 4);
    uint64_t n_t = 0, n_kv = 0;
    out.f.read((char*)&n_t, 8);
    out.f.read((char*)&n_kv, 8);

    const uint64_t kMaxKV = 200000, kMaxT = 200000;
    if (n_kv == 0 || n_kv > kMaxKV) {
        std::cerr << "ERROR: suspicious n_kv=" << n_kv << "\n";
        return false;
    }
    if (n_t == 0 || n_t > kMaxT) {
        std::cerr << "ERROR: suspicious n_t=" << n_t << "\n";
        return false;
    }

    // Skip KV
    try {
        for (uint64_t i = 0; i < n_kv; i++) {
            (void)rd_s(out.f);
            skipv(out.f);
        }
    }
    catch (const std::exception& e) {
        std::cerr << "WARN: KV skip failed: " << e.what() << ", continuing\n";
    }

    // Tensor table
    std::vector<GRec> recs;
    recs.reserve((size_t)n_t);
    for (uint64_t i = 0; i < n_t; i++) {
        std::string name = rd_s(out.f);
        if (!is_ascii_identifier(name)) {
            std::cerr << "ERROR: bad tensor name " << name << "\n";
            return false;
        }

        uint32_t nd = 0;
        out.f.read((char*)&nd, 4);
        if (nd == 0 || nd > 6) {
            std::cerr << "ERROR: bad nd=" << nd << " for " << name << "\n";
            return false;
        }

        std::vector<uint64_t> dims(nd);
        for (uint32_t d = 0; d < nd; ++d) {
            out.f.read((char*)&dims[d], 8);
            if (dims[d] == 0 || dims[d] > (uint64_t) 1e10) {
                std::cerr << "ERROR: bad dim[" << d << "]=" << dims[d] << " for " << name << "\n";
                return false;
            }
        }

        uint32_t g = 0;
        out.f.read((char*)&g, 4);
        uint64_t off = 0;
        out.f.read((char*)&off, 8);
        if (!aligned32(off)) {
            std::cerr << "ERROR: tensor data offset not 32B aligned for " << name << "\n";
            return false;
        }

        recs.push_back({ splitId, std::move(name), nd, std::move(dims), g, off, 0 });
    }

    // 4) Remember the start of the data block
    uint64_t data_block_start = static_cast<uint64_t>(out.f.tellg());
    data_block_start = align_up(data_block_start, 32);

    // Sizes by next off / EOF
    out.f.seekg(0, std::ios::end);
    size_t fsz = (size_t)out.f.tellg();

    std::vector<GRec*> ord;
    ord.reserve(recs.size());
    for (auto& r : recs) {
        ord.push_back(&r);
    }
    std::sort(ord.begin(), ord.end(), [](auto* a, auto* b) { return a->off < b->off; });
    for (size_t i = 0; i < ord.size(); ++i) {
        uint64_t nxt = (i + 1 < ord.size()) ? ord[i + 1]->off : (uint64_t)fsz;
        if (nxt < ord[i]->off) {
            std::cerr << "ERROR: decreasing offsets in baseline\n";
            return false;
        }
        ord[i]->sz = nxt - ord[i]->off;
    }

    for (auto& r : recs)
        r.off += data_block_start;

    // Whole file
    out.recs = std::move(recs);
    out.f.seekg(0, std::ios::beg);
    return true;
}

// =============== CUDA Dequant Kernels ===============
// Q8_0: per 32 values: [float scale][int8 x 32]
__global__ void k_deq_q8(const int8_t* q, const uint16_t* s, __half* out, int blocks) {
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (id >= blocks) {
        return;
    }

    __half         h = __ushort_as_half(s[id]);
    float          d = __half2float(h);

    if (!isfinite(d))
        d = 0.0f;

    const int8_t* src = q + id * QK;
    __half* dst = out + id * QK;
#pragma unroll
    for (int i = 0; i < QK; i++) {
        dst[i] = __float2half(d * float(src[i]));
    }
}

namespace q2k_detail {
    static constexpr int    QK_K = 256;
    static constexpr int    SCALES_SZ = QK_K / 16; // 16
    static constexpr int    QS_SZ = QK_K / 4;  // 64
    // CPU layout: [scales | qs | d(2) dmin(2)]
    static constexpr int    BYTES_PER_BLOCK = SCALES_SZ + QS_SZ + 4; // 84

    __device__ __forceinline__ uint16_t ld_u16_le(const uint8_t* p) {
        return static_cast<uint16_t>(p[0]) | (static_cast<uint16_t>(p[1]) << 8);
    }
}

// Kernel: writes __half
__global__ void k_deq_q2_k_to_f16_scales_qs_tail(
    const uint8_t* __restrict__ bytes,
    __half* __restrict__ out,
    size_t n_elems,
    int    blocks)
{
    using namespace q2k_detail;

    const int b = blockIdx.x * blockDim.x + threadIdx.x;
    if (b >= blocks) return;

    const size_t block_byte_off = static_cast<size_t>(b) * BYTES_PER_BLOCK;
    const uint8_t* base_ptr = bytes + block_byte_off;

    // [scales | qs | d(2) dmin(2)]
    const uint8_t* scales = base_ptr;                 // 16 bytes
    const uint8_t* qs = base_ptr + SCALES_SZ;     // 64 bytes
    const uint8_t* tail = qs + QS_SZ;               // 4 bytes

    const uint16_t d16 = q2k_detail::ld_u16_le(tail + 0);
    const uint16_t dmin16 = q2k_detail::ld_u16_le(tail + 2);

    const float d = __half2float(__ushort_as_half(d16));
    const float dmin = __half2float(__ushort_as_half(dmin16));

    const size_t out_base = static_cast<size_t>(b) * QK_K;
    if (out_base >= n_elems) return;
    const int block_elems = static_cast<int>(min(static_cast<size_t>(QK_K), n_elems - out_base));

#pragma unroll
    for (int g = 0; g < 16; ++g) {
        const uint8_t s = scales[g];
        const float dl = d * float(s & 0x0F);
        const float ml = dmin * float(s >> 4);

        const uint8_t b0 = qs[g * 4 + 0];
        const uint8_t b1 = qs[g * 4 + 1];
        const uint8_t b2 = qs[g * 4 + 2];
        const uint8_t b3 = qs[g * 4 + 3];

#pragma unroll
        for (int i = 0; i < 16; ++i) {
            const int idx_in_block = g * 16 + i;
            if (idx_in_block >= block_elems) break;

            const uint8_t pack = (i < 4) ? b0 : (i < 8) ? b1 : (i < 12) ? b2 : b3;
            const uint8_t shift = static_cast<uint8_t>((i & 3) * 2);
            const uint8_t q2 = static_cast<uint8_t>((pack >> shift) & 0x3u);

            // Match CPU: val = dl * q2 - ml
            const float val = dl * float(q2) - ml;

            out[out_base + idx_in_block] = __float2half_rn(val);
        }
    }
}


static __half* dequant_try_Q8_0(const uint8_t* bytes, size_t sz, size_t n_elems)
{
    const int blocks = int((n_elems + QK - 1) / QK);

    // Q8_0 => blocks * (2 + 32)
    size_t expect = size_t(blocks) * (2 + QK);
    if (sz == expect) {
        std::vector<int8_t>   hq(size_t(blocks) * QK);
        std::vector<uint16_t> hs(blocks);
        const uint8_t* p = bytes;
        for (int b = 0; b < blocks; b++) {
            uint16_t bits;
            memcpy(&bits, p, 2);
            p += 2;
            hs[b] = bits;
            memcpy(&hq[size_t(b) * QK], p, QK);
            p += QK;
        }
        int8_t* dq = nullptr;
        uint16_t* ds = nullptr;
        __half* out = nullptr;
        CUDA_OK(cudaMalloc(&dq, hq.size()));
        CUDA_OK(cudaMalloc(&ds, hs.size() * sizeof(uint16_t)));
        CUDA_OK(cudaMalloc(&out, n_elems * sizeof(__half)));
        CUDA_OK(cudaMemcpy(dq, hq.data(), hq.size(), cudaMemcpyHostToDevice));
        CUDA_OK(cudaMemcpy(ds, hs.data(), hs.size() * sizeof(uint16_t), cudaMemcpyHostToDevice));
        int th = 256, bl = (blocks + th - 1) / th;
        k_deq_q8 << <bl, th >> > (dq, ds, out, blocks);
        CUDA_OK(cudaDeviceSynchronize());
        cudaFree(dq);
        cudaFree(ds);
        return out;
    }
}
// Try to dequantize a quant payload by size signature; return device FP16 (caller cudaFree) or nullptr
static __half* dequant_try_Q2_K(const uint8_t* bytes, size_t sz, size_t n_elems)
{
    using namespace q2k_detail;

    if (!bytes || n_elems == 0) return nullptr;

    auto ceil_div = [](size_t a, size_t b) { return (a + b - 1) / b; };
    const int    blocks = int(ceil_div(n_elems, size_t(QK_K)));
    const size_t expect = size_t(blocks) * BYTES_PER_BLOCK;
    if (sz != expect) {
        // Not a raw Q2_K payload (or wrong size for QK_K=256).
        return nullptr;
    }

    uint8_t* d_bytes = nullptr;
    __half* d_out = nullptr;

    CUDA_OK(cudaMalloc(&d_bytes, sz));
    CUDA_OK(cudaMemcpy(d_bytes, bytes, sz, cudaMemcpyHostToDevice));
    CUDA_OK(cudaMalloc(&d_out, n_elems * sizeof(__half)));

    const int th = 256;
    const int bl = (blocks + th - 1) / th;
    k_deq_q2_k_to_f16_scales_qs_tail << <bl, th >> > (d_bytes, d_out, n_elems, blocks);
    //CUDA_OK(cudaGetLastError());
    CUDA_OK(cudaDeviceSynchronize());

    cudaFree(d_bytes);
    return d_out;  // device pointer; free with cudaFree
}

__global__ void k_f32_to_f16(__half* o, const float* x, size_t n)
{
    size_t i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n)
        o[i] = __float2half(x[i]);
}

// =============== Main ===============
int main(int argc, char** argv)
{
    std::string pmp, pfp;
    bool report = false, diffHL = false;

    for (int i = 1; i < argc; i++)
    {
        std::string a = argv[i];
        if (a == "--mpgguf" && i + 1 < argc)
            pmp = argv[++i];
        else if (a == "--fp16" && i + 1 < argc)
            pfp = argv[++i];
        else if (a == "--report")
            report = true;
        else if (a == "--diff-high-low")
            diffHL = true;
        else
        {
            std::cerr << "Usage: --mpgguf <file> --fp16 <f16_or_f32.gguf> [--report] [--diff-high-low]\n";
            return 1;
        }
    }
    if (pmp.empty() || pfp.empty())
    {
        std::cerr << "Missing --mpgguf or --fp16\n";
        return 1;
    }
    auto split = [](const std::string& s, char delimiter) -> std::vector<std::string>
    {
        std::vector<std::string> tokens;
        std::string token;
        std::istringstream tokenStream(s);
        while (std::getline(tokenStream, token, delimiter)) {
            tokens.push_back(token);
        }
        return tokens;
    };

    auto pfps = split(pfp, ',');

    MP mp;
    if (!load_mp(pmp, mp)) {
        std::cerr << "bad mpgguf\n";
        return 1;
    }
    std::vector<GG> ggs;
    ggs.resize(pfps.size());

    for (size_t i = 0; i < pfps.size(); i++)
    {
        if (!load_fp(i, pfps[i], ggs[i]))
        {
            std::cerr << "bad gguf baseline\n";
            return 1;
        }
    }

    // Map baseline by name
    std::unordered_map<std::string, GRec*> truth;
    for (auto& gg : ggs)
    {
        for (auto& r : gg.recs)
            truth[r.name] = &r;
    }

    auto N = [](const std::vector<uint64_t>& d) -> size_t {
        return numel(d);
        };
    const size_t kMaxElems = size_t(1) << 36;  // Defensive ~64B elements

    // Accumulators
    double s_low = 0, n_low = 0, s_high = 0, n_high = 0, s_hl = 0, n_hl = 0;
    float  m_low = 0, m_high = 0, m_hl = 0;
    size_t validated_high = 0, validated_low = 0;

    auto reduce = [&](const __half* A, const __half* B, size_t n, double& s, double& Nacc, float& m) {
        std::vector<__half> ha(n), hb(n);
        CUDA_OK(cudaMemcpy(ha.data(), A, n * sizeof(__half), cudaMemcpyDeviceToHost));
        CUDA_OK(cudaMemcpy(hb.data(), B, n * sizeof(__half), cudaMemcpyDeviceToHost));
        double ss = 0;
        float  mx = 0;
        for (size_t i = 0; i < n; i++) {
            float d = __half2float(ha[i]) - __half2float(hb[i]);

            if (std::isnan(d) || std::isinf(d))
                continue;

            ss += double(d) * double(d);
            float ad = fabsf(d);
            if (ad > mx) {
                mx = ad;
            }
        }
        s += ss;
        Nacc += double(n);
        if (mx > m) {
            m = mx;
        }
        };

    for (const auto& r : mp.recs)
    {
        auto it = truth.find(r.name);
        if (it == truth.end())
        {
            if (report)
                std::cerr << "SKIP (missing in baseline): " << r.name << "\n";
            continue;
        }
        const GRec& tr = *it->second;

        if (tr.dims != r.dims)
        {
            if (report)
                std::cerr << "SKIP (shape mismatch): " << r.name << "\n";
            continue;
        }

        size_t n = N(r.dims);
        if (n == 0 || n > kMaxElems)
        {
            if (report)
                std::cerr << "SKIP (absurd n=" << n << "): " << r.name << "\n";
            continue;
        }

        // Build FP16 truth (accept FP16 or FP32 tensor payloads)
        __half* d_truth = nullptr;
        //std::cout << tr.splitId << "," << tr.off << "\n";
        auto stream_data = readStreamData(ggs[tr.splitId].f, n * sizeof(float), tr.off);
        const uint8_t* tb = stream_data.data();
        //for (size_t i = 0; i < 100; i++)
        //    std::cout << (int)stream_data[i] << ",";
        //std::cout << r.name << "\n";

        if (tr.g == 30)
        {
            CUDA_OK(cudaMalloc(&d_truth, tr.sz));
            CUDA_OK(cudaMemcpy(d_truth, tb, tr.sz, cudaMemcpyHostToDevice));
        }
        else if (tr.g == 0)
        {
            float* df = nullptr;
            CUDA_OK(cudaMalloc(&df, tr.sz));
            CUDA_OK(cudaMemcpy(df, tb, tr.sz, cudaMemcpyHostToDevice));
            CUDA_OK(cudaMalloc(&d_truth, n * sizeof(__half)));
            int th = 256, bl = (int)((n + th - 1) / th);
            k_f32_to_f16 << <bl, th >> > (d_truth, df, n);
            CUDA_OK(cudaDeviceSynchronize());
            cudaFree(df);
        }
        else
        {
            // Unknown baseline tensor type; zero it so it doesn't pollute aggregates
            CUDA_OK(cudaMalloc(&d_truth, n * sizeof(__half)));
            CUDA_OK(cudaMemset(d_truth, 0, n * sizeof(__half)));
            if (report) {
                std::cerr << "WARN: baseline tensor " << r.name << " not FP16/FP32, treated as zeros\n";
            }
        }

        // HIGH (Q8_0 expected)
        if (r.sz_high && in_range(r.off_high, r.sz_high, mp.data_sz) && r.g_high == 8)
        {
            auto data_high = readStreamData(mp.f, r.sz_high, mp.data_offset + r.off_high);
            const uint8_t* hb = data_high.data();

            __half* d_high = dequant_try_Q8_0(hb, (size_t)r.sz_high, n);
            if (d_high)
            {
                reduce(d_high, d_truth, n, s_high, n_high, m_high);
                cudaFree(d_high);
                validated_high++;
                std::cout << "Tensor name for INT8_0: " << validated_high << ", " << r.name << "\n";

                if (validated_high > 100)
                  break;
            }
            else if (report)
            {
                std::cerr << "WARN: unrecognized HIGH payload for " << r.name << "\n";
            }
        }

        // LOW (legacy scalar 2-bit only; Q2_K/IQ2_* skipped)
        if (r.sz_low && in_range(r.off_low, r.sz_low, mp.data_sz) && r.g_low == 10)
        {
            auto data_low = readStreamData(mp.f, r.sz_low, mp.data_offset + r.off_low);
            const uint8_t* lb = data_low.data();
            __half* d_low = dequant_try_Q2_K(lb, (size_t)r.sz_low, n);
            if (d_low)
            {
                reduce(d_low, d_truth, n, s_low, n_low, m_low);
                cudaFree(d_low);
                validated_low++;
                std::cout << "Tensor name for INT2_K: " << validated_low << ", " << r.name << "\n";
            }
            else if (report)
                std::cerr << "NOTE: skipping LOW dequant (likely Q2_K / IQ2_*): " << r.name << "\n";
        }

        cudaFree(d_truth);
    }

    auto outStats = [&](const char* tag, double s, double Nacc, float m, size_t ok) {
        if (Nacc <= 0) {
            std::cout << tag << ": N=0 (no tensors validated)\n";
            return;
        }
        double mse = s / Nacc;
        double rmse = std::sqrt(mse);
        std::cout << tag << ": tensors=" << ok << "  N=" << (uint64_t)Nacc << "  MSE=" << mse << "  RMSE=" << rmse
            << "  max =" << m << "\n";
        };

    outStats("HIGH(Q8_0) vs FP16", s_high, n_high, m_high, validated_high);
    outStats("LOW(2-bit)  vs FP16", s_low, n_low, m_low, validated_low);
    if (diffHL)
        outStats("HIGH vs LOW", s_hl, n_hl, m_hl, std::min(validated_high, validated_low));

    CUDA_OK(cudaDeviceReset());
    return 0;
}


Writing validate_mpgguf.cu


In [4]:
# ! cp "/content/drive/MyDrive/Qwen3-1.7B-BF16.gguf" .
# ! cp "/content/drive/MyDrive/qwen3-1.7-2.mpgguf" .

In [5]:
!nvcc -O3 -o validate_mpgguf validate_mpgguf.cu

validate_mpgguf.cu(695): warning #940-D: missing return statement at end of non-void function "dequant_try_Q8_0"
  }
  ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

validate_mpgguf.cu(695): warning #940-D: missing return statement at end of non-void function "dequant_try_Q8_0"
  }
  ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

validate_mpgguf.cu(144): warning #177-D: function "rd_u32" was declared but never referenced
  static inline uint32_t rd_u32(const uint8_t* p)
                         ^

validate_mpgguf.cu(151): warning #177-D: function "rd_u64" was declared but never referenced
  static inline uint64_t rd_u64(const uint8_t* p)
                         ^

validate_mpgguf.cu: In function ‘__half* dequant_try_Q8_0(const uint8_t*, size_t, size_t)’:
validate_mpgguf.cu:695:1: warning: control reaches end of non-void function []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wreturn-type-Wretur

In [6]:
! ./validate_mpgguf --mpgguf /content/drive/MyDrive/Qwen3-30B-A3B.mpgguf --fp16 /content/drive/MyDrive/Qwen3-30B-A3B-BF16-00001-of-00002.gguf,/content/drive/MyDrive/Qwen3-30B-A3B-BF16-00002-of-00002.gguf

Tensor name for INT8_0: 1, output.weight
Tensor name for INT8_0: 2, token_embd.weight
Tensor name for INT2_K: 1, token_embd.weight
Tensor name for INT8_0: 3, blk.0.attn_k.weight
Tensor name for INT2_K: 2, blk.0.attn_k.weight
Tensor name for INT8_0: 4, blk.0.attn_output.weight
Tensor name for INT8_0: 5, blk.0.attn_q.weight
Tensor name for INT2_K: 3, blk.0.attn_q.weight
Tensor name for INT8_0: 6, blk.0.attn_v.weight
Tensor name for INT8_0: 7, blk.0.ffn_down_exps.weight
Tensor name for INT8_0: 8, blk.0.ffn_gate_exps.weight
Tensor name for INT2_K: 4, blk.0.ffn_gate_exps.weight
Tensor name for INT8_0: 9, blk.0.ffn_up_exps.weight
Tensor name for INT2_K: 5, blk.0.ffn_up_exps.weight
Tensor name for INT8_0: 10, blk.1.attn_k.weight
Tensor name for INT2_K: 6, blk.1.attn_k.weight
Tensor name for INT8_0: 11, blk.1.attn_output.weight
Tensor name for INT8_0: 12, blk.1.attn_q.weight
Tensor name for INT2_K: 7, blk.1.attn_q.weight
Tensor name for INT8_0: 13, blk.1.attn_v.weight
Tensor name for INT8_0: 1